<a href="https://colab.research.google.com/github/Darsangmdd/Fundamental-Analysis-EV-EDBITDA-model-/blob/main/NIFTY_50_EV_EBITDA_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import csv

In [2]:
main_source = requests.get('https://www.moneycontrol.com/stocks/marketstats/indexcomp.php?optex=NSE&opttopic=indexcomp&index=9').text

In [3]:
parser = BeautifulSoup(main_source, 'lxml')
links = []
table = parser.find("table", class_='tbldata14 bdrtpg')
for tr in table.find_all("tr")[1:]:
  links.append(tr.find_all('a')[0].attrs['href'])
links

['/india/stockpricequote/infrastructuregeneral/adaniportsspecialeconomiczone/MPS',
 '/india/stockpricequote/paintsvarnishes/asianpaints/AP31',
 '/india/stockpricequote/banksprivatesector/axisbank/AB16',
 '/india/stockpricequote/auto23wheelers/bajajauto/BA10',
 '/india/stockpricequote/financeleasinghirepurchase/bajajfinance/BAF',
 '/india/stockpricequote/financeinvestments/bajajfinserv/BF04',
 '/india/stockpricequote/telecommunicationsservice/bhartiairtel/BA08',
 '/india/stockpricequote/refineries/bharatpetroleumcorporation/BPC',
 '/india/stockpricequote/foodprocessing/britanniaindustries/BI',
 '/india/stockpricequote/pharmaceuticals/cipla/C',
 '/india/stockpricequote/miningminerals/coalindia/CI11',
 '/india/stockpricequote/pharmaceuticals/divislaboratories/DL03',
 '/india/stockpricequote/pharmaceuticals/drreddyslaboratories/DRL',
 '/india/stockpricequote/autolcvshcvs/eichermotors/EM',
 '/india/stockpricequote/diversified/grasimindustries/GI01',
 '/india/stockpricequote/computerssoftwar

In [4]:
dataset = pd.DataFrame(columns = ['Company', 'Target Price', 'Entry Price', 'LTP', '%Increase'])
dataset

,Company,Target Price,Entry Price,LTP,%Increase


In [5]:
for company in links:
  symbol = company.split('/')[-1]
  company_name = company.split('/')[-2]
  s1 = 'https://www.moneycontrol.com/financials/'+ company_name + '/consolidated-ratiosVI/' + symbol + '#' + symbol
  s2 = 'https://www.moneycontrol.com/financials/'+ company_name + '/consolidated-balance-sheetVI/' + symbol + '#' + symbol
  s3 = 'https://www.moneycontrol.com/' + company

  try:

    source = requests.get(s1).text
    soup = BeautifulSoup(source, 'lxml')
    tabs = []
    table = soup.find('table' , class_="mctable1")
    for tr in table.find_all('tr')[1:]:
              tabs.append({
                  'KEY RATIOS (in Rs. Cr.)': tr.find_all('td')[0].text,
                  'MAR 21': tr.find_all('td')[1].text,
                  'MAR 20': tr.find_all('td')[2].text,
                  'MAR 19': tr.find_all('td')[3].text,
                  'MAR 18': tr.find_all('td')[4].text,
                  'MAR 17': tr.find_all('td')[5].text,
                    })
        
    df = pd.DataFrame(tabs)
    df.set_index('KEY RATIOS (in Rs. Cr.)', inplace = True)

    df2 = df.loc[['Enterprise Value (Cr.)','EV/EBITDA (X)'],:]
    
    df2['MAR 21'] = df2['MAR 21'].str.replace(',','').astype('float')
    df2['MAR 20'] = df2['MAR 20'].str.replace(',','').astype('float')
    df2['MAR 19'] = df2['MAR 19'].str.replace(',','').astype('float')
    df2['MAR 18'] = df2['MAR 18'].str.replace(',','').astype('float')
    df2['MAR 17'] = df2['MAR 17'].str.replace(',','').astype('float')

    EBITDA = df2.iloc[0,:].values / df2.iloc[1,:].values
    EBITDA = EBITDA.tolist()
    df2.loc['EBITDA'] = EBITDA

    pd.set_option('display.float_format', '{:.2f}'.format)

    Growth_in_EBITDA = []
    for i in range(len(EBITDA)-1):
      value = ((EBITDA[i]-EBITDA[i+1])/EBITDA[i+1])*100
      Growth_in_EBITDA.append(value)
      i+=1 
    
    Expected_EBITDA = (EBITDA[0]*(100+np.mean(Growth_in_EBITDA[0:3])))/100
    Ratio = df2.iloc[1,0]

    source1 = requests.get(s2).text
    soup1 = BeautifulSoup(source1, 'lxml')
    table = soup1.find('table' , class_="mctable1")
    tabs = []
    for tr in table.find_all('tr')[1:]:
      tabs.append({
                  'BALANCE SHEET OF RELIANCE INDUSTRIES (in Rs. Cr.)': tr.find_all('td')[0].text,
                  'MAR 21': tr.find_all('td')[1].text,
                  'MAR 20': tr.find_all('td')[2].text,
                  'MAR 19': tr.find_all('td')[3].text,
                  'MAR 18': tr.find_all('td')[4].text,
                  'MAR 17': tr.find_all('td')[5].text,
                  })
        
    df3 = pd.DataFrame(tabs)
    df3.set_index('BALANCE SHEET OF RELIANCE INDUSTRIES (in Rs. Cr.)', inplace = True)

    df4 = df3.loc[['Long Term Borrowings'],:]
    df4['MAR 21'] = df4['MAR 21'].str.replace(',','').astype('float')
    Debt = df4.iloc[0,0]

    Forecasted_EV = (Expected_EBITDA*Ratio) - Debt

    source2 = requests.get(s3).text
    soup2 = BeautifulSoup(source2, 'lxml')
    table = soup2.find('table' , class_="peer_tbl")

    tabs = []
    for tr in table.find_all('tr')[1:]:
      tabs.append({
                  'Company Name': tr.find_all('td')[0].text,
                  'LTP': tr.find_all('td')[1].text,
                  'MCap': tr.find_all('td')[3].text,
                  })
      
    df5 = pd.DataFrame(tabs)
    df5.set_index('Company Name', inplace = True)
    df5['LTP'] = df5['LTP'].str.replace(',','').astype('float')
    df5['MCap'] = df5['MCap'].str.replace(',','').astype('float')

    Outstanding_shares = df5.iloc[0,1]/df5.iloc[0,0]
    LTP = df5.iloc[0,0]
    TP= (Forecasted_EV)/ Outstanding_shares
    EP = 0.75*TP

    Percentage_Increase = ((TP - LTP)/LTP)*100
  
  except Exception as e:
    print(company_name)
    print('======>>>>>> ERROR <<<<<<=======')
    print('----------------------------------')
    print()
    continue

  dataset = dataset.append({'Company': company_name, 'Target Price' : TP ,'Entry Price' : EP ,'LTP':LTP , '%Increase': Percentage_Increase }, ignore_index = True)

  print('Company: ' + company_name)
  print('Target Price: ', TP)
  print('Entry Price: ', EP)
  print('LTP: ', LTP)
  print('%Gain/Loss if we buy now: ', Percentage_Increase )
  print('----------------------------------')
  print()



Company: adaniportsspecialeconomiczone
Target Price:  782.4567370702572
Entry Price:  586.8425528026929
LTP:  712.2
%Gain/Loss if we buy now:  9.86474825473984
----------------------------------

Company: asianpaints
Target Price:  2912.0003974616548
Entry Price:  2184.000298096241
LTP:  3003.9
%Gain/Loss if we buy now:  -3.0593429387910827
----------------------------------

axisbank
======>>>>>> ERROR <<<<<<=======
----------------------------------

Company: bajajauto
Target Price:  3701.7456341110505
Entry Price:  2776.309225583288
LTP:  4196.15
%Gain/Loss if we buy now:  -11.782332993075775
----------------------------------

Company: bajajfinance
Target Price:  8990.409671638134
Entry Price:  6742.807253728601
LTP:  6075.8
%Gain/Loss if we buy now:  47.97079679446548
----------------------------------

Company: bajajfinserv
Target Price:  12884.192022545938
Entry Price:  9663.144016909453
LTP:  12486.6
%Gain/Loss if we buy now:  3.184149588726616
---------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in true_divide


Company: hdfclifeinsurancecompanylimited
Target Price:  nan
Entry Price:  nan
LTP:  725.95
%Gain/Loss if we buy now:  nan
----------------------------------

Company: heromotocorp
Target Price:  2710.169756591242
Entry Price:  2032.6273174434314
LTP:  2939.2
%Gain/Loss if we buy now:  -7.792264677761228
----------------------------------

Company: hindalcoindustries
Target Price:  359.57293792430823
Entry Price:  269.6797034432312
LTP:  375.9
%Gain/Loss if we buy now:  -4.343458918779394
----------------------------------

Company: hindustanunilever
Target Price:  2779.1175595044224
Entry Price:  2084.338169628317
LTP:  2448.65
%Gain/Loss if we buy now:  13.495908337427656
----------------------------------

icicibank
======>>>>>> ERROR <<<<<<=======
----------------------------------

indusindbank
======>>>>>> ERROR <<<<<<=======
----------------------------------

Company: infosys
Target Price:  1448.3234556322857
Entry Price:  1086.2425917242142
LTP:  1574.2
%Gain/Loss if we buy now

In [14]:
dataset.dropna(inplace=True)
dataset.set_index('Company', inplace = True)

In [16]:
dataset.sort_values(by='%Increase', ascending= False)

,Target Price,Entry Price,LTP,%Increase
Company,,,,
housingdevelopmentfinancecorporation,5570.90,4178.17,2509.20,122.02
indianoilcorporation,182.14,136.60,111.40,63.50
ntpc,173.46,130.10,116.05,49.47
bajajfinance,8990.41,6742.81,6075.80,47.97
bharatpetroleumcorporation,573.80,430.35,470.05,22.07
ultratechcement,8442.46,6331.85,6923.00,21.95
shreecements,34487.16,25865.37,28729.10,20.04
britanniaindustries,4401.88,3301.41,3670.05,19.94
upl,952.91,714.69,805.15,18.35


In [17]:
dataset.to_csv('NIFTY50_model.csv')